In [ ]:
#@title Import Modules
import io
import os
import shutil
import sys
import tempfile
import time
import urllib
import zipfile

from IPython.display import display
from IPython.display import HTML
import numpy as np
import pandas as pd

import tensorflow as tf

from IPython.display import display
from IPython.display import HTML
import numpy as np
import pandas as pd

import tensorflow.keras as keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing import text

import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
toxicity_data_url = ("https://github.com/conversationai/unintended-ml-bias-analysis/"
                     "raw/e02b9f12b63a39235e57ba6d3d62d8139ca5572c/data/")

data_train = pd.read_csv(toxicity_data_url + "wiki_train.csv")
data_test = pd.read_csv(toxicity_data_url + "wiki_test.csv")
data_vali = pd.read_csv(toxicity_data_url + "wiki_dev.csv")

data_train

,rev_id,toxicity,comment,year,logged_in,ns,sample,split,is_toxic
0,2232.0,0.1,This: :One can make an analogy in mathematical...,2002,True,article,random,train,False
1,4216.0,0.0,` :Clarification for you (and Zundark's righ...,2002,True,user,random,train,False
2,26547.0,0.0,`This is such a fun entry. Devotchka I once...,2002,True,article,random,train,False
3,37330.0,0.3,` I fixed the link; I also removed ``homeopa...,2002,True,article,random,train,False
4,37346.0,0.1,`If they are ``indisputable`` then why does th...,2002,True,article,random,train,False
...,...,...,...,...,...,...,...,...,...
95687,699822249.0,0.2,"` :``Comment````. Gentlemen, this article pro...",2016,True,article,blocked,train,False
95688,699826615.0,0.0,*Support and recommend moving this (and my re...,2016,True,article,random,train,False
95689,699843603.0,0.0,` == File:Romantic Warriors cover.jpg == You...,2016,True,user,random,train,False
95690,699848324.0,0.0,` These sources don't exactly exude a sense ...,2016,True,article,blocked,train,False


In [ ]:
# Labels
labels_train = data_train["is_toxic"].values.reshape(-1, 1) * 1.0
labels_test = data_test["is_toxic"].values.reshape(-1, 1) * 1.0
labels_vali = data_vali["is_toxic"].values.reshape(-1, 1) * 1.0

In [ ]:
terms = {
    'sexuality': ['gay', 'lesbian', 'bisexual', 'homosexual', 'straight', 'heterosexual'], 
    'gender identity': ['trans', 'transgender', 'cis', 'nonbinary'],
    'muslum-jew': ['muslim', 'jewish'], # 'christian', 'buddhist', 'catholic', 'protestant', 'sikh', 'taoist'
    'christian': ['christian'],
    'race': ['african', 'african american', 'black', 'white', 'european', 'hispanic', 'latino', 'latina', 
             'latinx', 'mexican', 'canadian', 'american', 'asian', 'indian', 'middle eastern', 'chinese', 
             'japanese']}

group_names = list(terms.keys())
num_groups = len(group_names)

In [ ]:
def get_groups(text):
    # Returns a boolean NumPy array of shape (n, k), where n is the number of comments, 
    # and k is the number of groups. Each entry (i, j) indicates if the i-th comment 
    # contains a term from the j-th group.
    groups = np.zeros((text.shape[0], num_groups))
    for ii in range(num_groups):
        groups[:, ii] = text.str.contains('|'.join(terms[group_names[ii]]), case=False)
    return groups

groups_train = get_groups(data_train["comment"])
groups_test = get_groups(data_test["comment"])
groups_vali = get_groups(data_vali["comment"])

In [ ]:
print("Overall label proportion = %.1f%%" % (labels_train.mean() * 100))

group_stats = []
for ii in range(num_groups):
    group_proportion = groups_train[:, ii].mean()
    group_pos_proportion = labels_train[groups_train[:, ii] == 1].mean()
    group_stats.append([group_names[ii],
                        "%.2f%%" % (group_proportion * 100), 
                        "%.1f%%" % (group_pos_proportion * 100)])
group_stats = pd.DataFrame(group_stats, 
                           columns=["Topic group", "Group proportion", "Label proportion"])
group_stats

Overall label proportion = 9.7%


,Topic group,Group proportion,Label proportion
0,sexuality,1.30%,37.0%
1,gender identity,5.34%,7.7%
2,muslum-jew,0.92%,10.0%
3,christian,0.72%,7.6%
4,race,4.82%,9.2%


In [ ]:
muslim_jews = groups_train[:, 2].shape

(95692, 5)

In [ ]:
# Stanford Predifined Model
zip_file_url = "http://nlp.stanford.edu/data/glove.6B.zip"
zip_file = urllib.request.urlopen(zip_file_url)
archive = zipfile.ZipFile(io.BytesIO(zip_file.read()))

In [ ]:
# Hyper-parameters
hparams = {
    "batch_size": 128,
    "cnn_filter_sizes": [128, 128, 128],
    "cnn_kernel_sizes": [5, 5, 5],
    "cnn_pooling_sizes": [2, 2, 5],
    "constraint_learning_rate": 0.01,
    "embedding_dim": 100,
    "embedding_trainable": False,
    "learning_rate": 0.005,
    "max_num_words": 10000,
    "max_sequence_length": 250
}

In [ ]:
# Tokenizer
tokenizer = text.Tokenizer(num_words=hparams["max_num_words"])
tokenizer.fit_on_texts(data_train["comment"])

def prep_text(texts, tokenizer, max_sequence_length):
    # Turns text into into padded sequences.
    text_sequences = tokenizer.texts_to_sequences(texts)
    return sequence.pad_sequences(text_sequences, maxlen=max_sequence_length)

text_train = prep_text(data_train["comment"], tokenizer, hparams["max_sequence_length"])
text_test = prep_text(data_test["comment"], tokenizer, hparams["max_sequence_length"])
text_vali = prep_text(data_vali["comment"], tokenizer, hparams["max_sequence_length"])

In [ ]:
embeddings_index = {}
glove_file = "glove.6B.100d.txt"

with archive.open(glove_file) as f:
    for line in f:
        values = line.split()
        word = values[0].decode("utf-8") 
        coefs = np.asarray(values[1:], dtype="float32")
        embeddings_index[word] = coefs

embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, hparams["embedding_dim"]))
num_words_in_embedding = 0
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        num_words_in_embedding += 1
        embedding_matrix[i] = embedding_vector

In [ ]:
print(embedding_matrix.shape)

(150978, 100)


In [ ]:
# Modeling
def create_model():
    model = keras.Sequential()

    # Embedding layer.
    embedding_layer = layers.Embedding(
        embedding_matrix.shape[0],
        embedding_matrix.shape[1],
        weights=[embedding_matrix],
        input_length=hparams["max_sequence_length"],
        trainable=hparams['embedding_trainable'])
    model.add(embedding_layer)

    # Convolution layers.
    for filter_size, kernel_size, pool_size in zip(
        hparams['cnn_filter_sizes'], hparams['cnn_kernel_sizes'],
        hparams['cnn_pooling_sizes']):

        conv_layer = layers.Conv1D(
            filter_size, kernel_size, activation='relu', padding='same')
        model.add(conv_layer)

        pooled_layer = layers.MaxPooling1D(pool_size, padding='same')
        model.add(pooled_layer)

    # Add a flatten layer, a fully-connected layer and an output layer.
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(1))

    return model

In [ ]:
# Torch Modeling
class TextClassifier(nn.ModuleList):
  
  def __init__(self, lam = 0.05):
      super(TextClassifier, self).__init__()

      # Embedding layer definition
      self.embedding = nn.Embedding.from_pretrained(torch.tensor(embedding_matrix), freeze=False)
      # self.embedding = nn.Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1], _weight=embedding_matrix) 
      # self.embedding.weight = embedding_matrix

      # Convolution layers definition
      self.conv_1 = nn.Conv1d(250, hparams['cnn_filter_sizes'][0], hparams['cnn_kernel_sizes'][0], padding='same').double()
      self.pool_1 = nn.MaxPool1d(hparams['cnn_pooling_sizes'][0]).double()

      self.conv_2 = nn.Conv1d(128, hparams['cnn_filter_sizes'][1], hparams['cnn_kernel_sizes'][1], padding='same').double()
      self.pool_2 = nn.MaxPool1d(hparams['cnn_pooling_sizes'][1]).double()

      self.conv_3 = nn.Conv1d(128, hparams['cnn_filter_sizes'][2], hparams['cnn_kernel_sizes'][2], padding='same').double()
      self.pool_3 = nn.MaxPool1d(hparams['cnn_pooling_sizes'][2]).double()
      
       # Fully connected layer definition
      self.fc1 = nn.Linear(640, 128).double()
      self.fc2 = nn.Linear(128, 1).double()

      # Fairness Parameters
      self.m = 2 # Binary Label
      self.k = 2 # Binary Sensitive Attribute

      self.lam = lam
      self.W = nn.Parameter(torch.zeros(self.k, self.m))
    
  def forward(self, x):

    # Sequence of tokes is filterd through an embedding layer
    x = self.embedding(x).double()
    
    # Convolution layer 1 is applied
    x1 = self.conv_1(x)
    x1 = torch.relu(x1)
    # print(x1.shape)
    x1 = self.pool_1(x1)
    # print(x1.shape)

    # Convolution layer 2 is applied
    x2 = self.conv_2(x1)
    print(x2.shape)
    x2 = torch.relu((x2))
    x2 = self.pool_2(x2)
    # print(x2.shape)

    # Convolution layer 3 is applied
    x3 = self.conv_3(x2)
    x3 = torch.relu(x3)
    x3 = self.pool_3(x3)
    # print(x3.shape)
    # Flatten
    x3 = torch.flatten(x3, 1)
    # x3 = x3.reshape(x3.shape[0] * x3.shape[1] * x3.shape[2], -1)
    # print(x3.shape)
    

    out1 = self.fc1(x3)
    # print(out1.shape)
    
    # Second fully connected layer that outputs our 10 labels
    out2 = self.fc2(out1)

    out = torch.sigmoid(out2)
      
    return out

  def fairness_regularizer(self, X, S):

    current_batch_size = X.shape[0]      
    summation = 0
    
    Y_hat = self.forward(X)

    for i in range(current_batch_size):

      # Binary output:
      term = Y_hat[i] * Y_hat[i] * torch.matmul(self.W, torch.t(self.W))
      summation += -torch.trace(term)
      
      term1 = Y_hat[i] * self.W
      term2 = torch.matmul(term1, torch.t(S[i]).unsqueeze(0))
      term3 = torch.matmul(term2, self.P_s_sqrt_inv)
      summation += 2 * torch.trace(term3) - 1

    return self.lam * summation

In [ ]:
model = TextClassifier()

current_batch = text_train[0:100]
XTorch = torch.from_numpy(current_batch)
print(XTorch)
model(XTorch).size()
# model(tex)

tensor([[   0,    0,    0,  ...,   18, 6240, 5074],
        [   0,    0,    0,  ...,  585,  124,  318],
        [4371, 2061, 7314,  ...,    1,  616,  127],
        ...,
        [   0,    0,    0,  ..., 1017,    4, 2003],
        [   0,    0,    0,  ..., 5237, 1934,  296],
        [   0,    0,    0,  ..., 5237, 1934,  296]], dtype=torch.int32)
torch.Size([100, 128, 50])
torch.Size([100, 128, 50])
torch.Size([100, 128, 25])
torch.Size([100, 128, 5])
torch.Size([100, 640])
torch.Size([100, 128])


torch.Size([100, 1])

In [ ]:
# Train
number_of_epochs = 5
batch_size = 128
number_of_data_points = text_train.shape[0]
learning_rate_min = 10e-4
learning_rate_max = 10^-4
number_of_batches = number_of_data_points // batch_size + 1
# Define optimizer
optimizer = optim.Adam([model.parameters(), model.W], lr=learning_rate_min)
for _ in range(number_of_epochs):

  for i in range(number_of_batches):
    # Current_batch
    start = i * batch_size
    end = (i+1) * batch_size
    if i != number_of_batches - 1:
      currentX = torch.from_numpy(text_train[start:end])
      currentY = torch.from_numpy(labels_train[start:end])
    else:
      currentX = torch.from_numpy(text_train[start:])
      currentY = torch.from_numpy(labels_train[start:])
    
      # Feed the model
      y_pred = model(currentX)
      
      # Loss calculation
      loss = F.binary_cross_entropy(y_pred, currentY) + model.fairness_regularizer(X_S, S)
      
      # Clean gradientes
      optimizer.zero_grad()
      
      # Gradients calculation
      loss.backward()
      
      model.W.grad.data.mul_(-learning_rate_max / learning_rate_min) # You can have \eta_w here 
      # Gradients update
      optimizer.step()
  


torch.Size([76, 128, 50])
torch.Size([76, 128, 50])
torch.Size([76, 128, 25])
torch.Size([76, 128, 5])
torch.Size([76, 640])
torch.Size([76, 128])
torch.Size([76, 128, 50])
torch.Size([76, 128, 50])
torch.Size([76, 128, 25])
torch.Size([76, 128, 5])
torch.Size([76, 640])
torch.Size([76, 128])
torch.Size([76, 128, 50])
torch.Size([76, 128, 50])
torch.Size([76, 128, 25])
torch.Size([76, 128, 5])
torch.Size([76, 640])
torch.Size([76, 128])
torch.Size([76, 128, 50])
torch.Size([76, 128, 50])
torch.Size([76, 128, 25])
torch.Size([76, 128, 5])
torch.Size([76, 640])
torch.Size([76, 128])
torch.Size([76, 128, 50])
torch.Size([76, 128, 50])
torch.Size([76, 128, 25])
torch.Size([76, 128, 5])
torch.Size([76, 640])
torch.Size([76, 128])


In [ ]:
# Test
x_test = torch.from_numpy(text_test[0:200])
y_test = torch.from_numpy(labels_test[0:200])
y_pred = model(x_test)
final_preds = y_pred > 0.5
test = y_test == 1

acc = final_preds == test
true_preds = acc.sum(axis=0)
print(true_preds / 200)



torch.Size([200, 128, 50])
torch.Size([200, 128, 50])
torch.Size([200, 128, 25])
torch.Size([200, 128, 5])
torch.Size([200, 640])
torch.Size([200, 128])
tensor([0.9750])
